In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
from keras.layers import Dropout
from keras import optimizers


from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score


import re
import math
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import missingno
import seaborn as sns
plt.style.use('seaborn-whitegrid')
%matplotlib inline

In [ ]:
# Before loading the following dataset, be sure to run the preprocessing ipython

In [ ]:
data = pd.read_csv("1_dataset_ML_ready.csv", index_col=0 )

In [ ]:
#take a sample for testing the code
import random
indecies = random.sample(range(len(data)), 500)
data = data.iloc[indecies]

In [ ]:
X = data.drop('readmitted', axis=1)
Y = data['readmitted']

In [ ]:
#Split the dataset into training testing
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=101)

In [ ]:
#Scale the data
from sklearn.preprocessing import StandardScaler
Scaler = StandardScaler()
X_train_scaled = Scaler.fit_transform(X_train)
X_test_scaled = Scaler.transform(X_test)

In [ ]:
# Use the sklearn GridSearch capability with a keras model
#Wrap keras into a scikitlearn model
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier


#Build a wrapper function that returns the keras model
def build_keras_model(n_hidden_layers = 3,
                      activation_first= 'relu', 
                      activation_last = 'sigmoid', 
                      activation_middle_1 = 'relu',
                      activation_middle_2 = 'relu',
                      activation_before_decision_layers = 'relu',
                      n_nodes_first_layer = 128,  
                      n_nodes_deep_layers = 256, 
                      n_nodes_before_decision_layers = 128,
                      optimizer="adam"):
    
    model = tf.keras.Sequential()

    model.add(layers.Dense(n_nodes_first_layer, activation=activation_first))
    for i in range(n_hidden_layers):
        # Adds a densely-connected layer with 64 units to the model:
        if i%2 == 0:            
            model.add(layers.Dense(n_nodes_deep_layers, activation=activation_middle_1))            
        else:
            model.add(layers.Dense(n_nodes_deep_layers, activation=activation_middle_2))
            
        model.add(layers.Dropout(0.3))
            
        
    model.add(layers.Dense(n_nodes_before_decision_layers, activation=activation_before_decision_layers))
    model.add(layers.Dropout(0.3))  
        
    # Last layer that (activation depends on the number of categories in the outcome):
    model.add(layers.Dense(2, activation= activation_last ))
#     sgd = SGD(lr=0.01, momentum=0.8, decay=0.0, nesterov=False)
#     model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])
    
    model.compile(optimizer=optimizer,
#               loss='categorical_crossentropy',
              loss='binary_crossentropy', 
              metrics=['accuracy'])
    
    return model


In [ ]:
from sklearn.model_selection import GridSearchCV
params = {
    'n_hidden_layers': [3, 5, 10, 15],
    'activation_first': ['relu', 'sigmoid'], 
    'activation_last': ['sigmoid'], 
    'activation_middle_1' : ['relu', 'sigmoid'],
    'activation_middle_2' : ['relu', 'sigmoid'],
    'activation_before_decision_layers' : ['relu', 'sigmoid'],
    'n_nodes_first_layer': [64, 128, 256],  
    'n_nodes_deep_layers': [128, 256], 
    'n_nodes_before_decision_layers': [128, 256],
    'optimizer':["adam", 'sgd']
}
gs_model = GridSearchCV(estimator=keras_model, param_grid=params,n_jobs= -1)
gs_model.fit(X_train_scaled, y_train)

In [ ]:
# In progress

